In [39]:
import time
import pdb

# Sudoku Solver by Evan Freeman
## Basic Techniques, Then Brute Force


Input must be a string of 81 characters
Blank cells may be filled with anything for the input.



For example:
    .94...13..............76..2.8..1.....32.........2...6.....5.4.......8..7..63.4..8
    
Which solves to:
    794582136268931745315476982689715324432869571157243869821657493943128657576394218


Here's the coordinates of every cell in the grid:

(0,0) (0,1) (0,2) (0,3) (0,4) (0,5) (0,6) (0,7) (0,8)
(1,0) (1,1) (1,2) (1,3) (1,4) (1,5) (1,6) (1,7) (1,8)
(2,0) (2,1) (2,2) (2,3) (2,4) (2,5) (2,6) (2,7) (2,8)
(3,0) (3,1) (3,2) (3,3) (3,4) (3,5) (3,6) (3,7) (3,8)
(4,0) (4,1) (4,2) (4,3) (4,4) (4,5) (4,6) (4,7) (4,8)
(5,0) (5,1) (5,2) (5,3) (5,4) (5,5) (5,6) (5,7) (5,8)
(6,0) (6,1) (6,2) (6,3) (6,4) (6,5) (6,6) (6,7) (6,8)
(7,0) (7,1) (7,2) (7,3) (7,4) (7,5) (7,6) (7,7) (7,8)
(8,0) (8,1) (8,2) (8,3) (8,4) (8,5) (8,6) (8,7) (8,8)


Big picture:
    1) Start with naked / hidden singles techniques. This is enough for many puzzle.
        -Use the "little numbers" technique, where each cell has a "bag" of possible numbers that gets updated.
        -We will need to store these numbers in a list, so we can iterate through them as we 
    2) When you get stuck, finish off with brute force, but only considering 

To do:
    1) Accept more input types
    2) Check for multiple solutions
    3) Implement sudoku strategies to speed up solve time:
        -Could still finish off with brute force, even if I just put in a few basic strategies
        -Do the 'little numbers' technique, where I keep track of the possibilites associated with each cell
    4) Make it display as it goes?
        -That means showing MILLIONS of iterations. Would have to animate SUPER FAST.
        -Or collect all those images and make a gif of it, after the fact.
        - Pre-complile??? I don't know what I'm talking about
    5) Just how brute force is my algorithm? I think it's slightly faster than just populating every blank with a guess,
        checking the whole puzzle, then updating the whole puzzle and checking again (or even just checking the parts effected
        by the update).
        -After all, my algorithm cuts off certian possibility spaces as it goes
        -Still pretty brute force
    6) YES, I SHOULD HAVE MADE THE BLANKS OBJECTS, MAYBE EVEN ASSOCIATED THEM WITH THE CELL OBJECT SOMEHOW. SUE ME!!!
        -also should have named them "blanks" instead of "b"
        -Actually, maybe just a dict would have been better
    7) Add another strategy.
    8) Maybe add doubles and triples
        
        
Thoughts:
    Brute force time to complete depends on a few things:
        1) Luck. Whether a given blank cell will end up being 1 or 9 makes a big difference, as we start at 1.
        2) Contradiction depth. How long, on average, we can randomly fill cells before creating a contradiction?

Here is my original, pretty brute force, method.

In [40]:
#Here is my original, pretty brute force, method
def solve_bf(puzzle):
    start_time = time.time()
    class Grid:

        def __init__(self, string):
            self.cells = [[x for x in string[0:9]],
                          [x for x in string[9:18]],
                          [x for x in string[18:27]],
                          [x for x in string[27:36]],
                          [x for x in string[36:45]],
                          [x for x in string[45:54]],
                          [x for x in string[54:63]],
                          [x for x in string[63:72]],
                          [x for x in string[72:81]]]


        # This function outputs the contents of the box containing the cell with coordinates i, j
        def box(self, i, j):
            #let's find the coordinate of the upper left cell in the box
            #We'll calculate the rest of the cell from there

            #box x coordinate
            x = i // 3 * 3
            #box y coordinate
            y = j // 3 * 3

            box = [self.cells[a][b] for a in [x, x+1, x+2] for b in [y, y+1, y+2]]
            return box


        #This function outputs the contents of the row containing the cell with coordinates i, j
        def row(self, i, j):
            row = [self.cells[i][y] for y in range(9)]
            return row


        #This function outputs the contents of the column containing the cell with coordinates i, j
        def column(self, i, j):
            column = [self.cells[x][j] for x in range(9)]
            return column


        #Displays the puzzle, as a single block of strings
        def display(self):
            print('')
            for x in self.cells:
                print(''.join(x))
            print('')


        #Displays the puzzle, broken up into lists
        def display_grid(self):
            print('')
            for x in self.cells:
                print(x)
            print('')
     
        
        
#Checks a given input (box, row, or column) for duplicates
#Could be any list really, but will only check for duplicates in 1-9 (As strings)
#Should this be part of the sudoku object? Doesn't act on the object, so I don't think so
    def check(thing):
        #First, remove any empty spaces
        clean_thing = []
        for x in thing:
            if x in ['1', '2', '3', '4', '5', '6', '7', '8', '9']:
                clean_thing.append(x)
        #Now check for duplicates
        if len(clean_thing) == len(set(clean_thing)):
            return True
        else:
            return False

#Here is the solution function. Takes us from the original puzzle to the solution.

    sudoku = Grid(puzzle)

    print('Here is the brute force solution result:')
    sudoku.display()      

    
    #Step 1: #First, generate a list of all blank spaces, along with their coordinates, in the format of ['.', i, j]
    b = []
    for i in range(9):
        for j in range(9):
            if sudoku.cells[i][j] not in ['1', '2', '3', '4', '5', '6', '7', '8', '9']:
                #so keep track of the cell we are going to fill, and it's coordinates
                b.append([sudoku.cells[i][j], i, j])

    #Initialize some variables
    i = 0    
    count = 0 

    #This is the engine that drives the solution 
    #In each scenario, we update both the list of blanks, and the sudoku grid itself
    #Keep going until our index hits the length of blanks (Which is to say, we're one step beyond)
    while i != len(b):
        count += 1
        
        #Scenario 1: blank number i is still blank. Start with 1
        if b[i][0] not in ['1', '2', '3', '4', '5', '6', '7', '8', '9']:
            b[i][0] = '1'
            sudoku.cells[b[i][1]][b[i][2]] = b[i][0]
        
        #Scenario 2: blank number i is at 9. So we've already tried all the options
        #So we need to clear it out and step back.
        #Also we skip the rest of the loop, becacuse we don't need to check for consistency
        #In fact, it would be bad to check for consistency, as we are guarenteed to trivially be consistent
        #This would lead to stepping forward, canceling out our step back, and ending up in an infinite loop
        elif b[i][0] == '9':
            b[i][0] = '.'
            sudoku.cells[b[i][1]][b[i][2]] = b[i][0]
            i -= 1
            continue
        
        #Scenario 3: There's some number 1-8 already plugged in. So we step forward by one.
        else:
            b[i][0] = str(int(b[i][0]) + 1)
            sudoku.cells[b[i][1]][b[i][2]] = b[i][0]
        
        #Now we check for consistency. If we are consistent, we'll step forward.
        #If not, we'll run through this same spot again.
        consistent = check(sudoku.row(b[i][1], b[i][2])) and check(sudoku.column(b[i][1], b[i][2])) and check(sudoku.box(b[i][1], b[i][2]))
        if consistent:
            i += 1

    #Format the solution as a string of 81 characters, like the input
    solution = ''.join([''.join(x) for x in sudoku.cells])

    
    sudoku.display()
    print(solution)
    print('\n')
    print(f'This sudoku was solved in {count} loops.')
    print('\n')
    print(f'--- This program took {time.time() - start_time} seconds to run. ---')
    print('\n')
    print('-'*200)
    print('\n')
    return solution

Here is my limited brute force solver, which only brute forces over the possiblities for each cell, not all of 1-9

In [41]:
#Here is my limited brute force solver, which only brute forces over the possiblities for each cell, not all of 1-9

def solve_lbf(puzzle):
    start_time = time.time()
    class Grid:

        def __init__(self, string):
            self.cells = [[x for x in string[0:9]],
                          [x for x in string[9:18]],
                          [x for x in string[18:27]],
                          [x for x in string[27:36]],
                          [x for x in string[36:45]],
                          [x for x in string[45:54]],
                          [x for x in string[54:63]],
                          [x for x in string[63:72]],
                          [x for x in string[72:81]]]


        # This function outputs the contents of the box containing the cell with coordinates i, j
        def box(self, i, j):
            #let's find the coordinate of the upper left cell in the box
            #We'll calculate the rest of the cell from there

            #box x coordinate
            x = i // 3 * 3
            #box y coordinate
            y = j // 3 * 3

            box = [self.cells[a][b] for a in [x, x+1, x+2] for b in [y, y+1, y+2]]
            return box


        #This function outputs the contents of the row containing the cell with coordinates i, j
        def row(self, i, j):
            row = [self.cells[i][y] for y in range(9)]
            return row


        #This function outputs the contents of the column containing the cell with coordinates i, j
        def column(self, i, j):
            column = [self.cells[x][j] for x in range(9)]
            return column


        #Displays the puzzle, as a single block of strings
        def display(self):
            print('')
            for x in self.cells:
                print(''.join(x))
            print('')


        #Displays the puzzle, broken up into lists
        def display_grid(self):
            print('')
            for x in self.cells:
                print(x)
            print('')
     
        
        
#Checks a given input (box, row, or column) for duplicates
#Could be any list really, but will only check for duplicates in 1-9 (As strings)
#Should this be part of the sudoku object? Doesn't act on the object, so I don't think so
    def check(thing):
        #First, remove any empty spaces
        clean_thing = []
        for x in thing:
            if x in ['1', '2', '3', '4', '5', '6', '7', '8', '9']:
                clean_thing.append(x)
        #Now check for duplicates
        if len(clean_thing) == len(set(clean_thing)):
            return True
        else:
            return False

#Here is the solution function. Takes us from the original puzzle to the solution.

    sudoku = Grid(puzzle)

    print('Here is the less brute force solution result:')
    sudoku.display()      

    
    #Step 1: #First, generate a list of all blank spaces, along with their coordinates, and possibilities, in the format of ['.', i, j, [possible numbers]]
    b = []
    for i in range(9):
        for j in range(9):
            if sudoku.cells[i][j] not in ['1', '2', '3', '4', '5', '6', '7', '8', '9']:
                                
                poss = ['1', '2', '3', '4', '5', '6', '7', '8','9']
                real_poss = []
                
                for x in poss:
                    if not (x in sudoku.column(i, j) or x in sudoku.row(i, j) or x in sudoku.box(i, j)):
                        real_poss.append(x)
                
                b.append([sudoku.cells[i][j], i, j, real_poss])
    
    
    
    #Step 2: Finish with brute force, if needed.
        #Only brute force through the possibilites, though.
        #3 Possibilities, just like the other one.
            #1) Nothing filled in yet -> Use the first possibility
            #2) The last possibility filled in -> step back to previous "blank"
            #3) Else -> try the next possibility
        #Note that we are guarenteed to have at least 2 possibilities, as the previous code would have filled
        #In the solution if there were only one possibility
    
    i = 0
    count = 0
    
    while i != len(b):
        count += 1
        
        #Scenario 1: blank number i is still blank. Start with the first possibility
        if b[i][0] == '.':
            b[i][0] = b[i][3][0]
            sudoku.cells[b[i][1]][b[i][2]] = b[i][0]
        
        #Scenario 2: blank number i is at the last possibility. So we've already tried all the options
        #So we need to clear it out and step back.
        #Also we skip the rest of the loop, becacuse we don't need to check for consistency
        #In fact, it would be bad to check for consistency, as we are guarenteed to trivially be consistent
        #This would lead to stepping forward, canceling out our step back, and ending up in an infinite loop
        elif b[i][0] == b[i][3][-1]:
            b[i][0] = '.'
            sudoku.cells[b[i][1]][b[i][2]] = b[i][0]
            i -= 1
            continue
        
        #Scenario 3: There's some non last possibility already plugged in. So we step forward by one.
        else:
            b[i][0] = b[i][3][b[i][3].index(b[i][0]) + 1] #This is inefficient, I should store which poss I'm on
            sudoku.cells[b[i][1]][b[i][2]] = b[i][0]
        
        #Now we check for consistency. If we are consistent, we'll step forward.
        #If not, we'll run through this same spot again.
        consistent = check(sudoku.row(b[i][1], b[i][2])) and check(sudoku.column(b[i][1], b[i][2])) and check(sudoku.box(b[i][1], b[i][2]))
        if consistent:
            i += 1
        
        
        
        
#Format the solution as a string of 81 characters, like the input
    solution = ''.join([''.join(x) for x in sudoku.cells])

    
    sudoku.display()
    print(solution)
    print('\n')
    print(f'This sudoku was solved in {count} loops.')
    print('\n')
    print(f'--- This program took {time.time() - start_time} seconds to run. ---')
    print('\n')
    print('-'*200)
    print('\n')
    return solution

Here is my updated solver

Current Techniques:

1) Naked Singles

2) Hidden Singles

3) Naked Doubles

4) Naked Triples

5) Finish with limited brute force if necessary


In [42]:
#Here is my updated solver

def solve(puzzle):
    start_time = time.time()
    class Grid:

        def __init__(self, string):
            self.cells = [[x for x in string[0:9]],
                          [x for x in string[9:18]],
                          [x for x in string[18:27]],
                          [x for x in string[27:36]],
                          [x for x in string[36:45]],
                          [x for x in string[45:54]],
                          [x for x in string[54:63]],
                          [x for x in string[63:72]],
                          [x for x in string[72:81]]]


        # This function outputs the contents of the box containing the cell with coordinates i, j
        def box(self, i, j):
            #let's find the coordinate of the upper left cell in the box
            #We'll calculate the rest of the cell from there

            #box x coordinate
            x = i // 3 * 3
            #box y coordinate
            y = j // 3 * 3

            box = [self.cells[a][b] for a in [x, x+1, x+2] for b in [y, y+1, y+2]]
            return box


        #This function outputs the contents of the row containing the cell with coordinates i, j
        def row(self, i, j):
            row = [self.cells[i][y] for y in range(9)]
            return row


        #This function outputs the contents of the column containing the cell with coordinates i, j
        def column(self, i, j):
            column = [self.cells[x][j] for x in range(9)]
            return column


        #Displays the puzzle, as a single block of strings
        def display(self):
            print('')
            for x in self.cells:
                print(''.join(x))
            print('')


        #Displays the puzzle, broken up into lists
        def display_grid(self):
            print('')
            for x in self.cells:
                print(x)
            print('')
     
        
        
#Checks a given input (box, row, or column) for duplicates
#Could be any list really, but will only check for duplicates in 1-9 (As strings)
#Should this be part of the sudoku object? Doesn't act on the object, so I don't think so
    def check(thing):
        #First, remove any empty spaces
        clean_thing = []
        for x in thing:
            if x in ['1', '2', '3', '4', '5', '6', '7', '8', '9']:
                clean_thing.append(x)
        #Now check for duplicates
        if len(clean_thing) == len(set(clean_thing)):
            return True
        else:
            return False

#Here is the solution function. Takes us from the original puzzle to the solution.

    sudoku = Grid(puzzle)

    print('Here is the mostly not brute force solution result:')
    sudoku.display()      

    
    #Step 1: #First, generate a list of all blank spaces, along with their coordinates, and possibilities, in the format of ['.', i, j, [possible numbers]]
    blanks = []
    for i in range(9):
        for j in range(9):
            if sudoku.cells[i][j] not in ['1', '2', '3', '4', '5', '6', '7', '8', '9']:
                                
                poss = ['1', '2', '3', '4', '5', '6', '7', '8','9']
                real_poss = []
                
                for x in poss:
                    if not (x in sudoku.column(i, j) or x in sudoku.row(i, j) or x in sudoku.box(i, j)):
                        real_poss.append(x)
                
                blanks.append([sudoku.cells[i][j], i, j, real_poss])

   
    # Updates all blanks with new information in the sudoku
    def update_blanks():
        for blank in blanks:
            for poss in blank[3][:]:
                if poss in sudoku.column(blank[1], blank[2]) or poss in sudoku.row(blank[1], blank[2]) or poss in sudoku.box(blank[1], blank[2]):
                    blank[3].remove(poss)
   

    # Fill in a blank if there is only a single possibility
    def naked_single():
        for i in range(len(blanks)):
            if len(blanks[i][3]) == 1:
                # Update the puzzle
                sudoku.cells[blanks[i][1]][blanks[i][2]] = blanks[i][3][0]
                # Delete that entry in the blanks
                del blanks[i]
                # Note that progress has been made this loop
                return True
        return False
   

    # Fill in when there is only one remaining place for a number in a row, column, or box.
    def hidden_single():
        # For each blank, see if it is the only number in it's row, column, or box that could contain a given number
        # Nuts, maybe I should have attached the possibilites to each cell..., some sort of object
        
        #For each blank:
            #1) for each possiblity
            #2) look in it's row. Is there any other cell which is blank and has that possibility? If not, fill it in
            #3) Look in it's column. Is there any other cell which is blank and has that possibility? If not, fill it in
            #4) Look in it's box. Is there any other cell which is blank and has that possibility? If not, fill it in
        for blank in blanks:
            #generate the subset of blanks that are in the same column, row, or box as our current blank
            
            #These have the same second coordinate
            blank_column = [other_blank for other_blank in blanks if other_blank[2] == blank[2] and other_blank != blank]
            
            #These have the same first coordinate
            blank_row = [other_blank for other_blank in blanks if other_blank[1] == blank[1] and other_blank != blank]
            
            #These have the same whole number when divided by 3
            blank_box = [other_blank for other_blank in blanks if int(other_blank[1])//3 == int(blank[1])//3 and int(other_blank[2])//3 == int(blank[2])//3 and other_blank != blank]
            
            #Iterate through each possibility. See if it is the only 
            other_column_poss = {num for other_poss in blank_column for num in other_poss[3]}
            other_row_poss = {num for other_poss in blank_row for num in other_poss[3]}
            other_box_poss = {num for other_poss in blank_box for num in other_poss[3]}
            
            for poss in blank[3]:
                if not poss in other_column_poss or not poss in other_row_poss or not poss in other_box_poss:
                    sudoku.cells[blank[1]][blank[2]] = poss
                    blanks.remove(blank)
                    return True
        return False
    
    #Just naked doubles
    #THIS IS A BIT INEFFICIENT, I'M GENERATING THESE LISTS BOTH ABOVE AND HERE
    #Probably not a big deal though, and makes the code more readable and the logic easier to write for me
    def naked_double():     
        
        for blank in blanks:
            #generate the subset of blanks that are in the same column, row, or box as our current blank
            
            #These have the same second coordinate
            blank_column = [other_blank for other_blank in blanks if other_blank[2] == blank[2] and other_blank != blank]
            
            #These have the same first coordinate
            blank_row = [other_blank for other_blank in blanks if other_blank[1] == blank[1] and other_blank != blank]
            
            #These have the same whole number when divided by 3
            blank_box = [other_blank for other_blank in blanks if int(other_blank[1])//3 == int(blank[1])//3 and int(other_blank[2])//3 == int(blank[2])//3 and other_blank != blank]
        
            #See if any other blank in the row, column, or box has identical possiblities, and is length 2. If so, remove from that column, row, or box.
            for other_blank in blank_column:
                if other_blank[3] == blank[3] and len(blank[3]) == 2:
                    for other_other_blank in blank_column:
                        if other_other_blank != other_blank:
                            for poss in blank[3]:
                                if poss in other_other_blank[3]:
                                    other_other_blank[3].remove(poss)
                                    return True
                            
            for other_blank in blank_row:
                if other_blank[3] == blank[3] and len(blank[3]) == 2:
                    for other_other_blank in blank_row:
                        if other_other_blank != other_blank:
                            for poss in blank[3]:
                                if poss in other_other_blank[3]:
                                    other_other_blank[3].remove(poss)
                                    return True
                            
            for other_blank in blank_box:
                if other_blank[3] == blank[3] and len(blank[3]) == 2:
                    for other_other_blank in blank_box:
                        if other_other_blank != other_blank:
                            for poss in blank[3]:
                                if poss in other_other_blank[3]:
                                    other_other_blank[3].remove(poss)
                                    return True
        return False
        
        
        
        #Naked triples
        #Recall that a naked triple means 3 in a subsection that all have EXACTLY and only members of a len 3 subset of possibilities
        #So {1, 2}, {1, 3}, and {2, 3} would form a naked triple
        #I may assume that there are no naked singles or doubles because of the previous code
        #Instead of going through each blank and generating 
        #THIS IS A BIT INEFFICIENT, I'M GENERATING THESE LISTS BOTH ABOVE AND HERE
        #Probably not a big deal though, and makes the code more readable and the logic easier to write for me
    def naked_triple():
        #Let's generate each column, row, and box, but only for blanks
        #Remember, these are copies, so alter the original items in b???
        #Just make a list, or maybe a dict
        
        
        column_blanks = [[blank for blank in blanks if blank[2] == i] for i in range(9)]
        row_blanks = [[blank for blank in blanks if blank[1] == i] for i in range(9)]     
        box_blanks = [[blank for blank in blanks if blank[1] // 3 == i and blank[2] // 3 == j] for i in range(3) for j in range(3)]
        
        for column in column_blanks:
            for blank1 in column:
                if 2 <= len(blank1[3]) <= 3:
                    for blank2 in column:
                        triple = set(blank1[3] + blank2[3])
                        if blank2 != blank1 and len(triple) == 3:
                            for blank3 in column:
                                if blank3 != blank1 and blank3 != blank2 and set(blank3[3]).issubset(triple):
                                    for blank4 in column:
                                        if blank4 != blank1 and blank4 != blank2 and blank4 != blank3:
                                            for poss in triple:
                                                if poss in blank4[3]:
                                                    blank4[3].remove(poss)
                                                    return True
        for row in row_blanks:
            for blank1 in row:
                if 2 <= len(blank1[3]) <= 3:
                    for blank2 in row:
                        triple = set(blank1[3] + blank2[3])
                        if blank2 != blank1 and len(triple) == 3:
                            for blank3 in row:
                                if blank3 != blank1 and blank3 != blank2 and set(blank3[3]).issubset(triple):
                                    for blank4 in row:
                                        if blank4 != blank1 and blank4 != blank2 and blank4 != blank3:
                                            for poss in triple:
                                                if poss in blank4[3]:
                                                    blank4[3].remove(poss)
                                                    return True      
        for box in box_blanks:
            for blank1 in box:
                if 2 <= len(blank1[3]) <= 3:
                    for blank2 in box:
                        triple = set(blank1[3] + blank2[3])
                        if blank2 != blank1 and len(triple) == 3:
                            for blank3 in box:
                                if blank3 != blank1 and blank3 != blank2 and set(blank3[3]).issubset(triple):
                                    for blank4 in box:
                                        if blank4 != blank1 and blank4 != blank2 and blank4 != blank3:
                                            for poss in triple:
                                                if poss in blank4[3]:
                                                    blank4[3].remove(poss)
                                                    return True      
        return False
            
            
            
                
    #Step 2: Loop through basic strategies:
        # Hidden and Naked Singles
        # Now with naked doubles, triples, and quads!
        # When a blank is solved in this way, remove it from the list of blanks
        # Make sure to update each cell's possibilities as you go
        # Don't do more advanced strategies if you don't have to
    
    ns_count = 0
    hs_count = 0
    nd_count = 0
    nt_count = 0
    progress = True
    while progress == True:
        
        prog1 = naked_single()
        update_blanks()        
        progress = prog1
        ns_count += prog1
        
        if progress == False:
            prog2 = hidden_single()
            update_blanks()        
            progress = progress or prog2
            hs_count += prog2

            if progress == False:
                prog3 = naked_double()
                update_blanks()
                progress = progress or prog3
                nd_count += prog3
                
                if progress == False:
                    prog4 = naked_triple()
                    update_blanks
                    progress = progress or prog4
                    nt_count += prog4
            
        
        
        
        
            
    
    sudoku.display()
    print(f'We solved {ns_count} cells with naked singles.')
    print(f'We solved {hs_count} cells with hidden singles.')
    print(f'We helped {nd_count} times with naked doubles.')
    print(f'We helped {nt_count} times with naked triples.')
    
    
    #Step 3: Finish with brute force, if needed.
        #Only brute force through the possibilites, though.
        #3 Possibilities, just like the other one.
            #1) Nothing filled in yet -> Use the first possibility
            #2) The last possibility filled in -> step back to previous "blank"
            #3) Else -> try the next possibility
        #Note that we are guarenteed to have at least 2 possibilities, as the previous code would have filled
        #In the solution if there were only one possibility

    i = 0
    count = 0
    
    while i != len(blanks):
        count += 1
        
        #Scenario 1: blank number i is still blank. Start with the first possibility
        if blanks[i][0] == '.':
            blanks[i][0] = blanks[i][3][0]
            sudoku.cells[blanks[i][1]][blanks[i][2]] = blanks[i][0]
        
        #Scenario 2: blank number i is at the last possibility. So we've already tried all the options
        #So we need to clear it out and step back.
        #Also we skip the rest of the loop, becacuse we don't need to check for consistency
        #In fact, it would be bad to check for consistency, as we are guarenteed to trivially be consistent
        #This would lead to stepping forward, canceling out our step back, and ending up in an infinite loop
        elif blanks[i][0] == blanks[i][3][-1]:
            blanks[i][0] = '.'
            sudoku.cells[blanks[i][1]][blanks[i][2]] = blanks[i][0]
            i -= 1
            continue
        
        #Scenario 3: There's some non last possibility already plugged in. So we step forward by one.
        else:
            blanks[i][0] = blanks[i][3][blanks[i][3].index(blanks[i][0]) + 1] #This is inefficient, I should store which poss I'm on
            sudoku.cells[blanks[i][1]][blanks[i][2]] = blanks[i][0]
        
        #Now we check for consistency. If we are consistent, we'll step forward.
        #If not, we'll run through this same spot again.
        consistent = check(sudoku.row(blanks[i][1], blanks[i][2])) and check(sudoku.column(blanks[i][1], blanks[i][2])) and check(sudoku.box(blanks[i][1], blanks[i][2]))
        if consistent:
            i += 1
        
        
        
        
#Format the solution as a string of 81 characters, like the input
    solution = ''.join([''.join(x) for x in sudoku.cells])

    
    sudoku.display()
    print(solution)
    print('\n')
    print(f'Brute force: {count} loops.')
    print('\n')
    print(f'--- This program took {time.time() - start_time} seconds to run. ---')
    print('-'*200)
    print('\n')
    return solution

In [43]:
#Random Sudoku

solve_bf('.94...13..............76..2.8..1.....32.........2...6.....5.4.......8..7..63.4..8')
solve_lbf('.94...13..............76..2.8..1.....32.........2...6.....5.4.......8..7..63.4..8')
solve('.94...13..............76..2.8..1.....32.........2...6.....5.4.......8..7..63.4..8')

Here is the brute force solution result:

.94...13.
.........
....76..2
.8..1....
.32......
...2...6.
....5.4..
.....8..7
..63.4..8


794582136
268931745
315476982
689715324
432869571
157243869
821657493
943128657
576394218

794582136268931745315476982689715324432869571157243869821657493943128657576394218


This sudoku was solved in 2329276 loops.


--- This program took 11.020208597183228 seconds to run. ---


--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


Here is the less brute force solution result:

.94...13.
.........
....76..2
.8..1....
.32......
...2...6.
....5.4..
.....8..7
..63.4..8


794582136
268931745
315476982
689715324
432869571
157243869
821657493
943128657
576394218

794582136268931745315476982689715324432869571157243869821657493943128657576394218


This sudoku was solved in 1219898 loops.


--- This program took 5.4

'794582136268931745315476982689715324432869571157243869821657493943128657576394218'

In [44]:
#Easiest Possible Sudoku, from sudoku wiki
#Only requires naked singles

solve_bf('...1.5...14....67..8...24...63.7..1.9.......3.1..9.52...72...8..26....35...4.9...')
solve_lbf('...1.5...14....67..8...24...63.7..1.9.......3.1..9.52...72...8..26....35...4.9...')
solve('...1.5...14....67..8...24...63.7..1.9.......3.1..9.52...72...8..26....35...4.9...')

Here is the brute force solution result:

...1.5...
14....67.
.8...24..
.63.7..1.
9.......3
.1..9.52.
..72...8.
.26....35
...4.9...


672145398
145983672
389762451
263574819
958621743
714398526
597236184
426817935
831459267

672145398145983672389762451263574819958621743714398526597236184426817935831459267


This sudoku was solved in 3115 loops.


--- This program took 0.019005298614501953 seconds to run. ---


--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


Here is the less brute force solution result:

...1.5...
14....67.
.8...24..
.63.7..1.
9.......3
.1..9.52.
..72...8.
.26....35
...4.9...


672145398
145983672
389762451
263574819
958621743
714398526
597236184
426817935
831459267

672145398145983672389762451263574819958621743714398526597236184426817935831459267


This sudoku was solved in 1190 loops.


--- This program took 0.00891

'672145398145983672389762451263574819958621743714398526597236184426817935831459267'

In [45]:
# Easy Puzzle 5,220,548,762 from Web Sudoku

solve_bf('.4.9....5.85.32.7....8...6.5.82.3..172..9..344..7.12.6.7...8....1.32.54.3....7.1.')
solve_lbf('.4.9....5.85.32.7....8...6.5.82.3..172..9..344..7.12.6.7...8....1.32.54.3....7.1.')
solve('.4.9....5.85.32.7....8...6.5.82.3..172..9..344..7.12.6.7...8....1.32.54.3....7.1.')

Here is the brute force solution result:

.4.9....5
.85.32.7.
...8...6.
5.82.3..1
72..9..34
4..7.12.6
.7...8...
.1.32.54.
3....7.1.


243976185
685132479
197854362
568243791
721695834
439781256
974518623
816329547
352467918

243976185685132479197854362568243791721695834439781256974518623816329547352467918


This sudoku was solved in 2773 loops.


--- This program took 0.019001245498657227 seconds to run. ---


--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


Here is the less brute force solution result:

.4.9....5
.85.32.7.
...8...6.
5.82.3..1
72..9..34
4..7.12.6
.7...8...
.1.32.54.
3....7.1.


243976185
685132479
197854362
568243791
721695834
439781256
974518623
816329547
352467918

243976185685132479197854362568243791721695834439781256974518623816329547352467918


This sudoku was solved in 926 loops.


--- This program took 0.008999

'243976185685132479197854362568243791721695834439781256974518623816329547352467918'

In [46]:
#More of a medium

solve_bf('.5247.....6............8.1.4.......97..95.....2..4..3....8...9......37.6....91...')
solve_lbf('.5247.....6............8.1.4.......97..95.....2..4..3....8...9......37.6....91...')
solve('.5247.....6............8.1.4.......97..95.....2..4..3....8...9......37.6....91...')

Here is the brute force solution result:

.5247....
.6.......
.....8.1.
4.......9
7..95....
.2..4..3.
...8...9.
.....37.6
....91...


152479683
368215974
974638512
416387259
783952461
529146837
237864195
891523746
645791328

152479683368215974974638512416387259783952461529146837237864195891523746645791328


This sudoku was solved in 3227386 loops.


--- This program took 14.39666223526001 seconds to run. ---


--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


Here is the less brute force solution result:

.5247....
.6.......
.....8.1.
4.......9
7..95....
.2..4..3.
...8...9.
.....37.6
....91...


152479683
368215974
974638512
416387259
783952461
529146837
237864195
891523746
645791328

152479683368215974974638512416387259783952461529146837237864195891523746645791328


This sudoku was solved in 1600652 loops.


--- This program took 7.60

'152479683368215974974638512416387259783952461529146837237864195891523746645791328'

In [47]:
#Kindof Hard 1

solve_bf('.....7....9...1.......45..6....2.....36...41.5.....8.9........4....18....815...32')
solve_lbf('.....7....9...1.......45..6....2.....36...41.5.....8.9........4....18....815...32')
solve('.....7....9...1.......45..6....2.....36...41.5.....8.9........4....18....815...32')

Here is the brute force solution result:

.....7...
.9...1...
....45..6
....2....
.36...41.
5.....8.9
........4
....18...
.815...32


653287941
794631258
128945376
819724563
236859417
547163829
965372184
372418695
481596732

653287941794631258128945376819724563236859417547163829965372184372418695481596732


This sudoku was solved in 54770833 loops.


--- This program took 238.3385329246521 seconds to run. ---


--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


Here is the less brute force solution result:

.....7...
.9...1...
....45..6
....2....
.36...41.
5.....8.9
........4
....18...
.815...32


653287941
794631258
128945376
819724563
236859417
547163829
965372184
372418695
481596732

653287941794631258128945376819724563236859417547163829965372184372418695481596732


This sudoku was solved in 23017404 loops.


--- This program took 97

'653287941794631258128945376819724563236859417547163829965372184372418695481596732'

In [48]:
#Greg's Evil Puzzle

solve_bf('.1...5...89..7.2...7.4.....5...1....3.8...7.6....4...9.....6.5...7.3..98...8...4.')
solve_lbf('.1...5...89..7.2...7.4.....5...1....3.8...7.6....4...9.....6.5...7.3..98...8...4.')
solve('.1...5...89..7.2...7.4.....5...1....3.8...7.6....4...9.....6.5...7.3..98...8...4.')

Here is the brute force solution result:

.1...5...
89..7.2..
.7.4.....
5...1....
3.8...7.6
....4...9
.....6.5.
..7.3..98
...8...4.


213965874
894371265
675482931
569718423
348529716
721643589
482196357
157234698
936857142

213965874894371265675482931569718423348529716721643589482196357157234698936857142


This sudoku was solved in 98209 loops.


--- This program took 0.4501028060913086 seconds to run. ---


--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


Here is the less brute force solution result:

.1...5...
89..7.2..
.7.4.....
5...1....
3.8...7.6
....4...9
.....6.5.
..7.3..98
...8...4.


213965874
894371265
675482931
569718423
348529716
721643589
482196357
157234698
936857142

213965874894371265675482931569718423348529716721643589482196357157234698936857142


This sudoku was solved in 41182 loops.


--- This program took 0.19501

'213965874894371265675482931569718423348529716721643589482196357157234698936857142'

In [49]:
#Naked double test puzzle

solve_bf('4.....938.32.941...953..24.37.6.9..4529..16736.47.3.9.957..83....39..4..24..3.7.9')
solve_lbf('4.....938.32.941...953..24.37.6.9..4529..16736.47.3.9.957..83....39..4..24..3.7.9')
solve('4.....938.32.941...953..24.37.6.9..4529..16736.47.3.9.957..83....39..4..24..3.7.9')

Here is the brute force solution result:

4.....938
.32.941..
.953..24.
37.6.9..4
529..1673
6.47.3.9.
957..83..
..39..4..
24..3.7.9


461572938
732894156
895316247
378629514
529481673
614753892
957248361
183967425
246135789

461572938732894156895316247378629514529481673614753892957248361183967425246135789


This sudoku was solved in 4485 loops.


--- This program took 0.03601813316345215 seconds to run. ---


--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


Here is the less brute force solution result:

4.....938
.32.941..
.953..24.
37.6.9..4
529..1673
6.47.3.9.
957..83..
..39..4..
24..3.7.9


461572938
732894156
895316247
378629514
529481673
614753892
957248361
183967425
246135789

461572938732894156895316247378629514529481673614753892957248361183967425246135789


This sudoku was solved in 1656 loops.


--- This program took 0.012982

'461572938732894156895316247378629514529481673614753892957248361183967425246135789'

In [50]:
#Naked Triple Test

solve_bf('...........19..5..56.31..9.1..6...28..4...7..27...4..3.4..68.35..2..59...........')
solve_lbf('...........19..5..56.31..9.1..6...28..4...7..27...4..3.4..68.35..2..59...........')
solve('...........19..5..56.31..9.1..6...28..4...7..27...4..3.4..68.35..2..59...........')

Here is the brute force solution result:

.........
..19..5..
56.31..9.
1..6...28
..4...7..
27...4..3
.4..68.35
..2..59..
.........


928547316
431986572
567312894
195673428
384251769
276894153
749168235
612435987
853729641

928547316431986572567312894195673428384251769276894153749168235612435987853729641


This sudoku was solved in 235300 loops.


--- This program took 1.0423600673675537 seconds to run. ---


--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


Here is the less brute force solution result:

.........
..19..5..
56.31..9.
1..6...28
..4...7..
27...4..3
.4..68.35
..2..59..
.........


928547316
431986572
567312894
195673428
384251769
276894153
749168235
612435987
853729641

928547316431986572567312894195673428384251769276894153749168235612435987853729641


This sudoku was solved in 109463 loops.


--- This program took 0.493

'928547316431986572567312894195673428384251769276894153749168235612435987853729641'

In [51]:
#World's Hardest Sudoku
#Impervious to basically all Sudoku strategies

solve_bf('8..........36......7..9.2...5...7.......457.....1...3...1....68..85...1..9....4..')
solve_lbf('8..........36......7..9.2...5...7.......457.....1...3...1....68..85...1..9....4..')
solve('8..........36......7..9.2...5...7.......457.....1...3...1....68..85...1..9....4..')

Here is the brute force solution result:

8........
..36.....
.7..9.2..
.5...7...
....457..
...1...3.
..1....68
..85...1.
.9....4..


812753649
943682175
675491283
154237896
369845721
287169534
521974368
438526917
796318452

812753649943682175675491283154237896369845721287169534521974368438526917796318452


This sudoku was solved in 495276 loops.


--- This program took 2.1790339946746826 seconds to run. ---


--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


Here is the less brute force solution result:

8........
..36.....
.7..9.2..
.5...7...
....457..
...1...3.
..1....68
..85...1.
.9....4..


812753649
943682175
675491283
154237896
369845721
287169534
521974368
438526917
796318452

812753649943682175675491283154237896369845721287169534521974368438526917796318452


This sudoku was solved in 241003 loops.


--- This program took 1.062

'812753649943682175675491283154237896369845721287169534521974368438526917796318452'

In [52]:
#Very Hard Benchmark 1

solve_bf('4.....8.5.3..........7......2.....6.....8.4......1.......6.3.7.5..2.....1.4......')
solve_lbf('4.....8.5.3..........7......2.....6.....8.4......1.......6.3.7.5..2.....1.4......')
solve('4.....8.5.3..........7......2.....6.....8.4......1.......6.3.7.5..2.....1.4......')

Here is the brute force solution result:

4.....8.5
.3.......
...7.....
.2.....6.
....8.4..
....1....
...6.3.7.
5..2.....
1.4......


417369825
632158947
958724316
825437169
791586432
346912758
289643571
573291684
164875293

417369825632158947958724316825437169791586432346912758289643571573291684164875293


This sudoku was solved in 97273649 loops.


--- This program took 462.8260164260864 seconds to run. ---


--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------


Here is the less brute force solution result:

4.....8.5
.3.......
...7.....
.2.....6.
....8.4..
....1....
...6.3.7.
5..2.....
1.4......


417369825
632158947
958724316
825437169
791586432
346912758
289643571
573291684
164875293

417369825632158947958724316825437169791586432346912758289643571573291684164875293


This sudoku was solved in 54959559 loops.


--- This program took 25

'417369825632158947958724316825437169791586432346912758289643571573291684164875293'